## Getting features out of a game string
I don't think this can be called feature 'extraction', in a proper sense, as there is a clear defined, domain specific way in which to extract features from our domain data. We probably should have used a library to do this, but we had extra time on our hands, and we had always kind of wanted to produce something that could play chess by ourselves. It is, I think, one of the standard programming projects that people undertake as a way of getting better at code. In addition, a chess library/engine, would have also already included a function for evaluating a board position/predicting a winner. So in a way, it either made sense to build everything from the ground up or not at all!

We don't think it is necessarily important to read this notebook through, but it is included here if one wants more explanation as to how we got things (sort of) working.

## Problems in reading the game string

In [48]:
import pandas as pd
df=pd.read_csv('data/cleanChess.csv')
game=df.loc[0]
game.moves

'd4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4'

The problem is how to turn this list of moves, into a set of different board positions. This form of notation is called PGN notation, and as we described in our presentation, it is intended to be read by humans, not computers. For an overview of PGN notation, see [wikipedia](https://en.wikipedia.org/wiki/Portable_Game_Notation). 

The main problem we encountered in reading the game string, is that it is fundamentally ambiguous unless you know the rules of chess.

![pin 1](imgs/pinknight1.png)

In the above image, the move to be made is NxF4. This means that a knight must capture the piece on F4. However, as we can see, two knights are in a position to make this capture.

![pin 2](imgs/pinknight2.png)

It would be immediately obvious to a human player that the left knight cannot move. If it was to move, the bishop would be able to capture white's king. This is considered 'illegal' in the rules of chess - you cannot put your king in a position where it may be captured. However, to understand this, the computer needs to know the movement patterns of both knights, the movement pattern of the bishop, and it needs an idea of the rules describing what is and what isn't legal.

Thus to read the game string, we needed what is called, in chess engine lingo, 'a board representation', a computational representation of the rules of chess.



## Representing the board

The first thing we needed was a way of representing the board. We used a nested array, with different numbers representing each piece, and positive/negative signs representing black and white. Thus a 1 represents a white pawn. A -1 represents a black pawn. A 3 represents a kinght. A 4 represents a bishop. A 5 represents a castle. A 9 represents a queen. A 1000 represents a king. It turns out that this is a very inefficient way to represent a chess board, but there wil be a short discussion of that at the end of this document.

Through out this notebook, color is represented by a single integer, either +1 for white, or -1 for black.

In [8]:
import print_board
board = board = [[5,3,4,1000,9,4,3,5],
    [1,1,1,0,1,1,1,1],
    [0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [-1,-1,-1,-1,-1,-1,-1,-1],
    [-5,-3,-4,-1000,-9,-4,-3,-5],]

print_board.display(board=board)

       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    p    p    p    p    p  


  6    _    _    _    _    _    _    _    _  


  5    _    _    _    _    _    _    _    _  


  4    _    _    _    _    P    _    _    _  


  3    _    _    _    _    _    _    _    _  


  2    P    P    P    P    _    P    P    P  


  1    R    N    B    Q    K    B    N    R  


![e4](imgs/e4.png)

## Representing the pieces

Then we needed a way to represent the pieces. We will only go into the details of two pieces. The Bishop and the King. Below is the code defining how a bishop can move. Please ignore the second part for now. It concerns what happens if the bishop is pinned, and to talk about that we will need to introduce the king class. 

In [8]:
from pieces.vc import validCoords
class bishop():

    def __init__(self):

        pass


    def returnValidMoves(self, x, y, color,board,pin):
        self.color = color
        arr = []
        Capturearr=[]
        Protectarr = []
        result={'moves':[], 'captures':[], 'protects':[]}
        if pin == None:

            #first part - how the piece moves if it isn't pineed

            for i in [[1,1], [-1, -1], [1,-1], [-1,1]]:

                ix = i[0]; iy = i[1]
                lx = x; ly = y
                blocked = False

                while not blocked:

                    lx+=ix; ly+=iy

                    if validCoords(lx,ly):

                        if board[lx][ly] == 0:

                            result['moves'].append([lx,ly])

                        elif board[lx][ly] * self.color < 0:

                            blocked = True
                            result['captures'].append([lx, ly])

                        else:

                            blocked = True
                            result['protects'].append([lx, ly])

                    else:
                        blocked = True

        else:
            
            #second part - how the piece moves if it has a 'pin' on it

            for i in [[1,1], [-1, -1], [1,-1], [-1,1]]:

                if i in pin:
                    ix = i[0]; iy = i[1]
                    lx = x; ly = y
                    blocked = False

                    while not blocked:

                        lx+=ix; ly+=iy

                        if validCoords(lx,ly):

                            if board[lx][ly] == 0:

                                result['moves'].append([lx,ly])

                            elif board[lx][ly] * self.color < 0:

                                blocked = True
                                result['captures'].append([lx, ly])

                            else:

                                blocked = True
                                result['protects'].append([lx, ly])

                        else:
                            blocked = True


        return result

![bishop move](imgs/bishopmove.png)

In [11]:
b=bishop()
b.returnValidMoves(x=0, y=2, color=1, board, pin=None)

{'captures': [],
 'moves': [[1, 3], [2, 4], [3, 5], [4, 6], [5, 7]],
 'protects': [[1, 1]]}

The method delivers a list of squares that the piece can 'move' to, a list of squares where it can capture a piece, and list of squares where it is 'protecting' a friendly piece (it can recapture if this piece is captured).

We can see here that the bishop can only move on certain squares on its left diagonal. It can also protect one pawn on its right diagonal. It is unable to capture anything. As you can see, to do this the bishop class only needs its x and y coordinates, its color, and a representation of the board on which it resides.

Next is the king

In [4]:
from pieces.vc import validCoords

class king():
    #haven't added a method to check for moves causing check
    #same as there isn't a method to check for pins

    #need to think of an efficient method for checking these two things

    def __init__(self):

        pass



    def canMove(self, origin, destination):

        pass

    def returnValidMoves(self, x, y, color, board, checkSquares=None):
        #using checkSquares resolves illegal moves, but not pins.
        arr = []
        Capturearr = []
        Protectarr = []
        self.color = color
        #print('color: ', color, '\n', 'checksquares:',checkSquares)


        for i in [[1,0], [-1, 0], [0,-1], [0,1],[1,1], [-1, -1], [1,-1], [-1,1]]:

            ix = i[0]; iy = i[1]
            lx = x+ix; ly = y+iy
            
            if validCoords(lx,ly):

                if  [lx,ly] not in checkSquares:
                    #print(board[lx][ly])
                    if board[lx][ly] == 0:
                        arr.append([lx,ly])
                        #print(True)

                    elif board[lx][ly]*self.color<0:
                        Capturearr.append([lx,ly])

                    else:
                        Protectarr.append([lx,ly])




        return {'moves':arr, 'captures':Capturearr, 'protects':Protectarr}
    

    def isChecked(self, x, y, checkedSquares):

        if [x,y] in checkedSquares:

            return True

        else:
            return False

    def returnPins(self, x, y, color,board):
        #sloppy method for finding pinned pieces

        pins={}


        #check verticals
        for i in [[1,0], [-1, 0], [0,-1], [0,1]]:

            ix = i[0]; iy = i[1]
            lx = x; ly = y
            Protected = False



            while True:

                lx+=ix; ly+=iy

                if validCoords(lx,ly):
                    if board[lx][ly]*color>=1:

                        if Protected:
                            break

                        else:
                            Protected = True
                            pin=[lx,ly]

                    elif board[lx][ly]*color<0:

                        if abs(board[lx][ly]) in [5,9] and Protected:

                            if i in [[1,0],[-1,0]]:
                                okdirections = [[1,0],[-1,0]]
                            else:
                                okdirections = [[0,-1], [0,1]]

                            pins[str(pin[0])+str(pin[1])]=okdirections
                            break
                        else:
                            break

                else:

                    break

        #check diagonals
        for i in [[1,1], [-1, -1], [1,-1], [-1,1]]:

            ix = i[0]; iy = i[1]
            lx = x; ly = y
            Protected = False


            while True:

                lx+=ix; ly+=iy

                if validCoords(lx, ly):
                    if board[lx][ly]*color>=1:

                        if Protected:
                            break

                        else:
                            Protected = True
                            pin=[lx,ly]

                    elif board[lx][ly]*color<0:

                        if abs(board[lx][ly]) in [4,9] and Protected:

                            if i in [[1,1], [-1, -1]]:
                                okdirections = [[1,1], [-1, -1]]
                            else:
                                okdirections = [[1,-1], [-1,1]]

                            pins[str(pin[0])+str(pin[1])]=okdirections
                            break

                        else:
                            break

                else:


                    break

        return pins


We are only considering the first part - the 'valid moves' method for now. It is similar to the bishops valid moves method. The difference is that the king can move vertically as well as diagonally, and its range is limited to only one square at a time. In addition, the king needs another variable 'checkSquares'. These are the squares to which the king is not able to move.

In [30]:
import print_board
board = board = [[5,3,4,1000,9,4,3,5],
    [1,1,1,0,1,1,1,1],
    [0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [-4,0,0,0,0,0,0,0],
    [0,0,0,0,-1,0,0,0],
    [-1,-1,-1,-1,0,-1,-1,-1],
    [-5,-3,-4,-1000,-9,-4,-3,-5],]

print_board.display(board=board)

       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    _    p    p    p    p  


  6    _    _    _    p    _    _    _    _  


  5    _    _    _    _    _    _    _    b  


  4    _    _    _    _    P    _    _    _  


  3    _    _    _    _    _    _    _    _  


  2    P    P    P    P    _    P    P    P  


  1    R    N    B    Q    K    B    N    R  


![bishop king](imgs/bishopking2.png)

In [65]:
k = king()
k.returnValidMoves(0,3, 1, board, checkSquares=[[1,3]])

{'captures': [], 'moves': [], 'protects': [[0, 2], [0, 4], [1, 4], [1, 2]]}

The square directly in front of the King ([1,3]) is a checked square. It cannot legally move there, so it goes in the checkedsquares array. The checked squares array is created by first calculating all of the squares to which the other pieces can legally move. 

In this position, the king can't legally move. However it is still protecting pieces on the squares adjacent to it

The king also has an isChecked method. It takes as an argument, a list of all the squares that the other player can legally move to. If the king is on one of these squres, then the king is considered to be in check. This is one of the features in our eventual output.

Now we look at the issue of 'pins', pieces that can't move, because doing so would put their king in check.

In [31]:
import print_board
board = board = [[5,3,4,1000,9,4,0,5],
    [1,1,1,0,0,1,1,1],
    [0,0,0,0,0,4,0,0],
    [0,0,0,0,0,0,0,0],
    [-4,0,0,1,1,0,0,-4],
    [0,0,0,0,-1,0,0,0],
    [-1,-1,-1,-1,0,-1,-1,-1],
    [-5,-3,-4,-1000,-9,-4,-3,-5],]

print_board.display(board=board)

       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    _    p    p    p    p  


  6    _    _    _    p    _    _    _    _  


  5    b    _    _    P    P    _    _    b  


  4    _    _    _    _    _    _    _    _  


  3    _    _    B    _    _    _    _    _  


  2    P    P    P    _    _    P    P    P  


  1    R    _    B    Q    K    B    N    R  


![pinned bishop](imgs/pinnedbishop.png)

In [7]:
#Now here white's left knight cannot move. It is effectively pinned. The king has a method for that
k.returnPins(0,3, 1, board)

{'25': [[1, 1], [-1, -1]]}

As you see it returns the square that is pinned '25' - [2,5], and the direction in which it is actually allowed to still move. The direction is described as an increment. 

Evaluating the bishop we get...

In [12]:
b.returnValidMoves(2,5, 1, board, pin=[[1, 1], [-1, -1]])

{'captures': [[4, 7]], 'moves': [[3, 6], [1, 4]], 'protects': [[0, 3]]}

We see that now the bishop can't move on its right diagonal at all, as it has been given this 'pin' restricting its movements.

The pieces do get a little bit more complicated. Pawns have slightly more complex movements, and kings are able to 'castle', but as this depends on both the kings and the castles, it is checked by a different method. We won't go into this here.

## The 'gamestate' object

All the pieces and the board are contained inside a 'gamestate' object

In [8]:
import gs
ep={-1:[], 1:[]}

canCastle = {1000:{'queen':True, 'king':True}, -1000:{'queen':True, 'king':True}}
hasCastled = {1000:False, -1000:False}

gamestate = gs.gamestate(board, enpassants = ep, canCastle = canCastle, hasCastled=hasCastled)
gamestate.collection

{-1000: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 3],
   'protects': []}],
 -9: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 4],
   'protects': []}],
 -5: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 0],
   'protects': []},
  {'captures': [], 'moves': [], 'pin': None, 'pos': [7, 7], 'protects': []}],
 -4: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [4, 0],
   'protects': []},
  {'captures': [], 'moves': [], 'pin': None, 'pos': [4, 7], 'protects': []},
  {'captures': [], 'moves': [], 'pin': None, 'pos': [7, 2], 'protects': []},
  {'captures': [], 'moves': [], 'pin': None, 'pos': [7, 5], 'protects': []}],
 -3: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 1],
   'protects': []},
  {'captures': [], 'moves': [], 'pin': None, 'pos': [7, 6], 'protects': []}],
 -1: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [6, 0],
   'protects': []},
  {'captures': [], 'moves': [], 'pin': None, 'pos

The gamestate has a collection variable, that tracks information for all of the pieces on the board - thier 'pos' or board coordinates, their 'moves', their 'captures', their 'pin', and their protects. Right now most of these variables are empty except for 'pos'. Running the below methods. will update them.

In [9]:
gamestate.getPinnedSquares()
gamestate.pinPieces()
gamestate.getAllMoves()
gamestate.collection

{-1000: [{'captures': [],
   'castles': {'king': False, 'queen': False},
   'moves': [],
   'pin': None,
   'pos': [7, 3],
   'protects': [[6, 3], [7, 2], [7, 4], [6, 2]]}],
 -9: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 4],
   'protects': [[6, 4], [7, 3], [7, 5], [6, 3], [6, 5]]}],
 -5: [{'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 0],
   'protects': [[6, 0], [7, 1]]},
  {'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 7],
   'protects': [[6, 7], [7, 6]]}],
 -4: [{'captures': [[0, 4]],
   'moves': [[5, 1], [3, 1], [2, 2], [1, 3]],
   'pin': None,
   'pos': [4, 0],
   'protects': [[6, 2]]},
  {'captures': [[2, 5]],
   'moves': [[3, 6], [5, 6]],
   'pin': None,
   'pos': [4, 7],
   'protects': [[6, 5]]},
  {'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 2],
   'protects': [[6, 1], [6, 3]]},
  {'captures': [],
   'moves': [],
   'pin': None,
   'pos': [7, 5],
   'protects': [[6, 4], [6, 6]]}],
 -3: [{'captures': [],
   

By finding out the pinned squares, placing pins on the pieces, and then looking for all of the valid moves, the collection is updated to return a dictionary object describing what every piece on the board can/can't do.

## Reading in games

Now that we have defined everything that is invalid, we can effectively parse the PGN notation. First though, we need a function to translate a 'move' into an instruction, and then into a new game state!

In [12]:
def MO(move):
    ''' "preprocesses" moves so they can be interpreted by the game state object'''
    places=['a','b','c','d','e','f','g']
    rows={'1':0,'2':1,'3':2,'4':3,'5':4,'6':5,'7':6,'8':7}
    pieces=['N','K','B','R','Q']
    cols = {'a':7, 'b': 6, 'c':5, 'd':4, 'e':3, 'f':2, 'g':1, 'h':0}
    pieces = {'N':3, 'B':4, 'Q':9, 'K':1000, 'R':5}
    move=list(move)
    new_move = {'cols':[], 'rows':[], 'piece':[], 'type':'', 'castles':[]}

    for key in list(move):

        if key in rows:

            new_move['rows'].append(rows[key])

        elif key in cols:

            new_move['cols'].append(cols[key])

        elif key in pieces:

            new_move['piece'].append(pieces[key])



        elif key=='=':

            new_move['type']='promote'



        elif key=='O':

            new_move['castles'].append('O')
            new_move['type']='castle'


    return new_move

In [13]:
MO('e4')

{'castles': [], 'cols': [3], 'piece': [], 'rows': [3], 'type': ''}

In [14]:
MO('Nxd4')

{'castles': [], 'cols': [4], 'piece': [3], 'rows': [3], 'type': ''}

In [15]:
MO('O-O-O')

{'castles': ['O', 'O', 'O'],
 'cols': [],
 'piece': [],
 'rows': [],
 'type': 'castle'}

It returns a brief dictionary describing the columns that are referenced in the move, the rows that are referenced, the type of the move, and if it is a 'castling' move.

This must be turned into an 'instruction' and then a new move. This is done inside the gamestate object. It is not particularly complicated, but we implemented it so inefficiently that there is a lot of code and we won't go into it here. Instead we'll just show how it works.

In [16]:
board = board = [[5,3,4,1000,9,4,3,5],
    [1,1,1,1,1,1,1,1],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [-1,-1,-1,-1,-1,-1,-1,-1],
    [-5,-3,-4,-1000,-9,-4,-3,-5],]
print_board.display(board=board)
#values for enpassants, and castling rights, have to be set at the beginning of a game.
ep={-1:[], 1:[]}
canCastle = {1000:{'queen':True, 'king':True}, -1000:{'queen':True, 'king':True}}
hasCastled = {1000:False, -1000:False}
#create a gamestate
gamestate = gs.gamestate(board, enpassants = ep, canCastle = canCastle, hasCastled=hasCastled)
#interpret the state of the board and find all possible moves
gamestate.makeCollection()
gamestate.getPinnedSquares()
gamestate.pinPieces()
gamestate.getAllMoves()



       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    p    p    p    p    p  


  6    _    _    _    _    _    _    _    _  


  5    _    _    _    _    _    _    _    _  


  4    _    _    _    _    _    _    _    _  


  3    _    _    _    _    _    _    _    _  


  2    P    P    P    P    P    P    P    P  


  1    R    N    B    Q    K    B    N    R  


We can now make a move 

In [17]:
#make a move
new_board = gamestate.moveToInstruction(color=1, move=MO('e4'))
print_board.display(board=new_board)

       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    p    p    p    p    p  


  6    _    _    _    _    _    _    _    _  


  5    _    _    _    _    _    _    _    _  


  4    _    _    _    _    P    _    _    _  


  3    _    _    _    _    _    _    _    _  


  2    P    P    P    P    _    P    P    P  


  1    R    N    B    Q    K    B    N    R  


A new gamestate is then created with this new board.

In [18]:
gamestate = gamestate.returnNewGameState(new_board)

We can then play through a whole game...

In [19]:
board = board = [[5,3,4,1000,9,4,3,5],
    [1,1,1,1,1,1,1,1],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [-1,-1,-1,-1,-1,-1,-1,-1],
    [-5,-3,-4,-1000,-9,-4,-3,-5],]
#values for enpassants, and castling rights, have to be set at the beginning of a game. 
ep={-1:[], 1:[]}
canCastle = {1000:{'queen':True, 'king':True}, -1000:{'queen':True, 'king':True}}
hasCastled = {1000:False, -1000:False}
gamestate = gs.gamestate(board, enpassants = ep, canCastle = canCastle, hasCastled=hasCastled)
color = 1
print_board.display(board=board)
print('\n\n\n')
    
moves='d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4'.split(' ')

#play through every move in the game string

for move in moves:
    print(move, '\n\n')
    gamestate.makeCollection()
    gamestate.getPinnedSquares()
    gamestate.pinPieces()
    gamestate.getAllMoves()
    new_board = gamestate.moveToInstruction(color=color, move=MO(move))
    gamestate = gamestate.returnNewGameState(new_board)
    print_board.display(board=new_board)
    print('\n\n------------------------------------------------\n\n')
    #flip the player color every time
    color = color*-1



       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    p    p    p    p    p  


  6    _    _    _    _    _    _    _    _  


  5    _    _    _    _    _    _    _    _  


  4    _    _    _    _    _    _    _    _  


  3    _    _    _    _    _    _    _    _  


  2    P    P    P    P    P    P    P    P  


  1    R    N    B    Q    K    B    N    R  




d4 


       A    B    C    D    E    F    G        


  8    r    n    b    q    k    b    n    r  


  7    p    p    p    p    p    p    p    p  


  6    _    _    _    _    _    _    _    _  


  5    _    _    _    _    _    _    _    _  


  4    _    _    _    P    _    _    _    _  


  3    _    _    _    _    _    _    _    _  


  2    P    P    P    _    P    P    P    P  


  1    R    N    B    Q    K    B    N    R  


------------------------------------------------


d5 


       A    B    C    D    E    F    G        


  8    r    n    b  

## Derived features

So far so good. But how do we get the features that we end up with in our eventual dataset? 

Well, mainly we just count things, like the sum total of pieces on the board, sum the amount of captures, look for instances of pawns being stacked (bad) or pawns in lines(good). Inside our 'stats' file, the analyze class just iterates through the board, and the data held in the gamestate object, and turns it into a list of features.

I will briefly explore how the features that proved most important were created.

### Basic score

This is just the sum of all the material on the board. It can be got pretty easily with the sum() method. As black's material is negative, and white's material is positive, this score tends towards zero if the game is evenly balanced.

In [21]:
board = gamestate.board
basicScore = sum([sum(row) for row in board])
print(basicScore)

1


### Possible moves

This is the difference between the number of moves available for white and the number of moves available for black. As you see below, it evenly balances out.

In [23]:
possible_moves = 0
for pieceType in gamestate.collection:
    color = int(pieceType/abs(pieceType))
    for piece in gamestate.collection[pieceType]:
        possible_moves += len(piece['moves'] * color)
        
print(possible_moves)

0


### Captures

This is the (balanced) sum of all the material that can be captured at anyone time.

In [26]:
captures = 0
for pieceType in gamestate.collection:
    color = int(pieceType/abs(pieceType))
    for piece in gamestate.collection[pieceType]:
        
        for capture in piece['captures']:
            captures += gamestate.board[capture[0]][capture[1]]
print(captures)
    
    

0


As you can see, the basic method for getting all of our derived features just involves iterating through the gamestate/board and summing values that have already been calculated. Below, we packaged them all into one big 'stats' object, that can analyze a gamestate, and return a row in a pandas dataframe describing the gamestate

## Deriving multiple features en masse / Mining games

### Derived features for a single board position

In [27]:
from stats import analyze as a
gamestate.makeCollection()
gamestate.getPinnedSquares()
gamestate.pinPieces()
gamestate.getAllMoves()
analysisObject = a()

In [28]:
row=analysisObject.analyze(move=15,gs=gamestate,RETURN=True)

The resulting dataframe represents one position occurring in a game

In [29]:
row.describe().T

,count,mean,std,min,25%,50%,75%,max
pawnsGuardingKings,1.0,3.0,NaN,3.0,3.0,3.0,3.0,3.0
possible_moves,1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
captures,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
forks,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
basicScore,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
protects,1.0,8.0,NaN,8.0,8.0,8.0,8.0,8.0
pins,1.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0
centrePawns,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
kingMoves,1.0,-3.0,NaN,-3.0,-3.0,-3.0,-3.0,-3.0
pawnRanks,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0


The features 00, 01.... 77 record the value of each square on the board by the way.

### Derived features for a whole game

We then packaged all of this into a 'miner', that can take a single row from our dataset and explode into into a dataframe of every position that arises in the game by taking every move in the game and turning it into a new gamestate that is then analyzed.

In [76]:
import miner
game=miner.mine(df.loc[0])

The resulting data frame represents all of the board positions that occurred in a single game

In [77]:
game.describe().T

,count,mean,std,min,25%,50%,75%,max
00,13.0,5.000000,0.000000,5.0,5.0,5.0,5.0,5.0
01,13.0,1.846154,1.519109,0.0,0.0,3.0,3.0,3.0
02,13.0,4.000000,0.000000,4.0,4.0,4.0,4.0,4.0
03,13.0,1000.000000,0.000000,1000.0,1000.0,1000.0,1000.0,1000.0
04,13.0,9.000000,0.000000,9.0,9.0,9.0,9.0,9.0
05,13.0,3.692308,1.109400,0.0,4.0,4.0,4.0,4.0
06,13.0,2.307692,1.315587,0.0,3.0,3.0,3.0,3.0
07,13.0,5.000000,0.000000,5.0,5.0,5.0,5.0,5.0
10,13.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
11,13.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


### Derived features for a whole dataset

To mine the whole data set then, we just do this in a loop untill we have a really large dataframe

In [ ]:
#Do not run this code! It will take forever!!!
df_all=miner.mine(df.loc[0])
for index in range(1, df.shape[0]):
    
    df_all.append(miner.mine(df.loc[index]))

That's about it. There was some complexity involved in managing castling, enpassants, and pieces being pinned. We have covered the 'pins' issue, as this is one of the main reasons for ambiguity in the gamestring, and why we wrote the miner in the first place. We won't go into the rest here as we don't feel it is particularly relevant to a data analytics class!

It should be noted that our code is seriously inefficient. 'Real' chess engines work with much lower level code. We probably should have used one of them, but it became something of a pet project to try and do it ourselves from the ground up.

## The Chess engine question

A chess engine that can 'play chess' basically consists of three things - a board representation that defines the rules of chess, an evaluation function that can say whether a board state is good or bad, and a search algorithm that can look several moves ahead, to determine the best paths of play.

We already had a basic 'board representation' in order to derive our features. The bulk of this assignment involved building a (flawed) model out of those features. It seemed a shame to not go the last step, and implement the full 'chess engine', even if it is very slow! Our code is so unefficient though, that we can only reasonably look two moves ahead. In addition, we found our model to be weakest in the early 'opening' stage of the game, so we mapped the first 20 positions of each game in our dataset to a kind of tree in a JSON file, and we have the bot play through those until the game diverges from what is stored in the tree.

We have stored a pickled model with a 'predict' method inside our forest class.


### Making predictions on a board position/gamestate

In [39]:
print_board.display(board=gamestate.board)

       A    B    C    D    E    F    G        


  8    r    n    b    q    k    _    n    r  


  7    p    p    _    _    _    _    p    p  


  6    _    _    p    _    p    _    _    _  


  5    b    _    _    _    _    _    _    _  


  4    _    _    _    P    _    B    _    _  


  3    _    _    N    _    _    N    _    _  


  2    P    P    _    _    P    P    P    P  


  1    R    _    _    Q    K    B    _    R  


In [38]:
import forest
model = forest.forest(from_pikl=True)

It can take a gamestate and the number of moves passed in the game, and return a win probability for white. Though the model is either a Logistic Regression, or a Random Forest Classifier, we use the 'predict_probaba' methods, as obviously not at all moves are as good as each other.

In [37]:
score = model.score(gamestate, moveNum=25)
print(score)

0.7


### Choosing the best move for a given gamestate

Then we have a thinker class, that will consider every possible move, score the moves, and return the best one. The thinker class automatically loads a model as shown above. 

We set the search depth here to 1, meaning that it will consider every possible move, and every possible response to that move by the opposite player. It will return the move for which the other player has the lowest scoring response...

We use a fairly standard implementation of the min max algorithm (if we are searching white moves then we are trying to maximize our score, if we are searching black moves then we are trying to minimize it), though to make it run faster and cut down the size of the search tree, we have included a small random chance that a move/branch will not be searched.

In [48]:
import thinker2
bot=thinker2.thinker()

In [49]:
move=bot.rootthink(gs=gamestate, color=-1,moveNum=25,searchdepth=1)
print(move)

checked
checked
checked
win prob: 0.2
win prob: 0.2
win prob: 0.1
win prob: 0.0
win prob: 0.2
win prob: 0.1
win prob: 0.2
win prob: 0.2
win prob: 0.0
win prob: 0.1
win prob: 0.1
win prob: 0.0
win prob: 0.1
win prob: 0.2
win prob: 0.0
win prob: 0.2
win prob: 0.2
win prob: 0.2
win prob: 0.2
win prob: 0.1
win prob: 0.0
win prob: 0.1
win prob: 0.1
win prob: 0.2
win prob: 0.1
win prob: 0.1
win prob: 0.1
win prob: 0.0
win prob: 0.2
win prob: 0.1
win prob: 0.0
Best move: {'type': 'simple', 'origin': [4, 7], 'destination': [2, 5]} win prob: 0.2
{'type': 'simple', 'origin': [4, 7], 'destination': [2, 5]}


Then we simply have to make this move...

In [51]:
board=gamestate.simpleMove(1,gamestate.board[move['origin'][0]][move['origin'][1]], move['origin'], move['destination'])
gamestate=gamestate.returnNewGameState(board)
print_board.display(board=gamestate.board)

       A    B    C    D    E    F    G        


  8    r    n    b    q    k    _    n    r  


  7    p    p    _    _    _    _    p    p  


  6    _    _    p    _    p    _    _    _  


  5    _    _    _    _    _    _    _    _  


  4    _    _    _    P    _    B    _    _  


  3    _    _    _    _    _    N    _    _  


  2    P    P    _    _    P    P    P    P  


  1    R    _    _    Q    K    B    _    R  


## (flawed) Proof of concept

We have packaged our chess bot into a simple flask app. It can be played by running the file 'run.py' in our src directory and then opening a browser window at localhost:5000. It uses the [chessboard.js](https://chessboardjs.com) front end for user interaction. 

It must be said that it is still super buggy. Clicking on a piece and not moving it will cause the chess bot to think that you have already made a move. There is a particularly pernicious bug where the chess bot will think it can castle twice, and effectively double its number of kings. Over all, it plays chess rather badly. It will sometimes exhibit a good run of play, and then make a move so desperately bad that we will feel embarrased for it.

We feel that the poor performance of our chess bot provides some interesting perspective on our underlying models, and the field of data analytics overall. E.g, a 70% accuracy score is not very good at all, when one poorly evaluated move can completely wreck a game of chess. A 90% accuracy score is not particularly good, when, with some models, a decision based on the model can wreck peoples' lives. In addition, the bot considers moves that a human would never play, and sometimes scores them quite well, which might say something about 'overfitting' in regard to our random forest model.